# 🎤 K-pop Chatbot - Chạy trên Google Colab

Notebook này hướng dẫn chạy Streamlit UI của chatbot trên Google Colab.


## Bước 1: Clone Repository và Cài đặt Dependencies


In [ ]:
# Clone repository (nếu chưa có)
# !git clone https://github.com/your-username/Social-network-analyst.git
# %cd Social-network-analyst

# Cài đặt dependencies
!pip install streamlit pyngrok -q
!pip install transformers torch sentence-transformers -q
!pip install networkx faiss-cpu -q


## Bước 2: Cấu hình ngrok (Lần đầu tiên)

Nếu chưa có ngrok authtoken:
1. Đăng ký tại: https://dashboard.ngrok.com/signup
2. Lấy token từ: https://dashboard.ngrok.com/get-started/your-authtoken
3. Chạy cell dưới và thay YOUR_TOKEN


In [ ]:
# Cấu hình ngrok authtoken (chỉ cần làm 1 lần)
# Uncomment và thay YOUR_TOKEN bằng token của bạn:

# from pyngrok import ngrok
# ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")

print("ℹ️  Nếu đã cấu hình ngrok trước đó, có thể bỏ qua bước này.")


## Bước 3: Chạy Streamlit với ngrok


In [ ]:
import subprocess
import threading
import time
from pyngrok import ngrok
import os
import sys

# Đảm bảo đang ở đúng thư mục
if 'Social-network-analyst' in os.getcwd():
    os.chdir('Social-network-analyst')

print("📂 Thư mục hiện tại:", os.getcwd())

# Kiểm tra file Streamlit app
streamlit_app_path = "src/chatbot/streamlit_app.py"
if not os.path.exists(streamlit_app_path):
    print(f"❌ Không tìm thấy: {streamlit_app_path}")
    print("💡 Hãy đảm bảo đã clone repository và đang ở đúng thư mục.")
else:
    print(f"✅ Tìm thấy: {streamlit_app_path}")


In [ ]:
# Hàm chạy Streamlit trong background
def run_streamlit():
    subprocess.run([
        sys.executable, "-m", "streamlit", "run",
        streamlit_app_path,
        "--server.port", "8501",
        "--server.address", "0.0.0.0",
        "--server.headless", "true",
        "--browser.gatherUsageStats", "false"
    ])

# Khởi động Streamlit trong thread riêng
print("🚀 Đang khởi động Streamlit...")
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# Đợi Streamlit khởi động (cần thời gian để load model)
print("⏳ Đang đợi Streamlit khởi động (có thể mất 30-60 giây)...")
time.sleep(10)

# Tạo ngrok tunnel
print("🔗 Đang tạo ngrok tunnel...")
try:
    public_url = ngrok.connect(8501, bind_tls=True)
    
    print("\n" + "="*70)
    print("✅ STREAMLIT ĐÃ SẴN SÀNG!")
    print("="*70)
    print(f"\n🌐 URL công khai: {public_url}")
    print(f"\n💡 Copy URL trên và mở trong trình duyệt để sử dụng chatbot.")
    print("\n⚠️  Lưu ý:")
    print("   - URL này sẽ thay đổi mỗi lần chạy lại")
    print("   - Để dừng: Chạy cell 'Dừng Server' bên dưới")
    print("="*70 + "\n")
    
except Exception as e:
    print(f"❌ Lỗi khi tạo ngrok tunnel: {e}")
    print("\n💡 Thử:")
    print("   1. Cấu hình ngrok authtoken (xem Bước 2)")
    print("   2. Kiểm tra port 8501 có đang được dùng không")
    print("   3. Đợi thêm vài giây rồi chạy lại cell này")


## Bước 4: Sử dụng Chatbot

1. Copy URL từ output trên
2. Mở URL trong trình duyệt (tab mới)
3. Sử dụng chatbot như bình thường


## Dừng Server


In [ ]:
# Dừng ngrok tunnel và Streamlit
from pyngrok import ngrok

try:
    ngrok.kill()
    print("✅ Đã dừng ngrok tunnel")
except:
    pass

# Kill Streamlit process
try:
    import psutil
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        if 'streamlit' in ' '.join(proc.info['cmdline'] or []):
            proc.kill()
            print(f"✅ Đã dừng Streamlit (PID: {proc.info['pid']})")
except:
    print("ℹ️  Để dừng Streamlit, restart runtime hoặc kill process thủ công")

print("\n💡 Hoặc restart runtime: Runtime > Restart runtime")
